In [1]:
import logging
import numpy as np
import torch
from EduCDM import CDM
from torch import nn
import torch.nn.functional as F
from tqdm import tqdm
import pandas as pd
from sklearn.metrics import roc_auc_score, accuracy_score

def irf(theta, a, b, c, D=1.702, *, F=np):
    return c + (1 - c) / (1 + F.exp(-D * a * (theta - b)))

irt3pl = irf

class FairnessLoss(nn.Module):
    def __init__(self, alpha=1.0, reference_position=6):
        super(FairnessLoss, self).__init__()
        self.alpha = alpha
        self.reference_position = reference_position
        self.eps = 1e-7

    def calculate_position_difference(self, predictions):
        batch_size = predictions.size(0)
        n_positions = predictions.size(1)

        sorted_indices = torch.argsort(predictions, dim=1, descending=True)
        position_mapping = torch.zeros_like(sorted_indices)

        for i in range(batch_size):
            position_mapping[i][sorted_indices[i]] = torch.arange(n_positions).to(predictions.device)

        ref_differences = self.reference_position - position_mapping
        return ref_differences

    def calculate_pairwise_probabilities(self, ref_differences):
        n_positions = ref_differences.size(1)

        diff_i = ref_differences.unsqueeze(2).expand(-1, -1, n_positions)
        diff_j = ref_differences.unsqueeze(1).expand(-1, n_positions, -1)

        relative_diff = diff_i - diff_j
        pred_probs = torch.sigmoid(self.alpha * relative_diff)
        return pred_probs

    def calculate_oracle_probabilities(self, ref_differences):
        n_positions = ref_differences.size(1)

        diff_i = ref_differences.unsqueeze(2).expand(-1, -1, n_positions)
        diff_j = ref_differences.unsqueeze(1).expand(-1, n_positions, -1)

        oracle_probs = (diff_i > diff_j).float()
        return oracle_probs

    def forward(self, predictions, targets):
        pred_differences = self.calculate_position_difference(predictions)
        true_differences = self.calculate_position_difference(targets)

        pred_probs = self.calculate_pairwise_probabilities(pred_differences)
        oracle_probs = self.calculate_oracle_probabilities(true_differences)

        n_positions = predictions.size(1)
        mask = torch.ones((n_positions, n_positions), device=predictions.device)
        mask[self.reference_position-1] = 0
        mask[:, self.reference_position-1] = 0
        mask.fill_diagonal_(0)
        mask = mask.bool()

        pred_probs = torch.clamp(pred_probs, self.eps, 1 - self.eps)
        loss = -(oracle_probs * torch.log(pred_probs) +
                 (1 - oracle_probs) * torch.log(1 - pred_probs))

        loss = loss * mask.unsqueeze(0)
        loss = loss.sum() / (mask.sum() * predictions.size(0))

        return loss

class IRTNet(nn.Module):
    def __init__(self, user_num, item_num, value_range, a_range, irf_kwargs=None):
        super(IRTNet, self).__init__()
        self.user_num = user_num
        self.item_num = item_num
        self.irf_kwargs = irf_kwargs if irf_kwargs is not None else {}

        self.theta = nn.Embedding(self.user_num, 1)
        self.a = nn.Embedding(self.item_num, 1)
        self.b = nn.Embedding(self.item_num, 1)
        self.c = nn.Embedding(self.item_num, 1)

        self.value_range = value_range
        self.a_range = a_range

    def forward(self, user, item):
        theta = torch.squeeze(self.theta(user), dim=-1)
        a = torch.squeeze(self.a(item), dim=-1)
        b = torch.squeeze(self.b(item), dim=-1)
        c = torch.squeeze(self.c(item), dim=-1)

        c = torch.sigmoid(c)

        if self.value_range is not None:
            theta = self.value_range * (torch.sigmoid(theta) - 0.5)
            b = self.value_range * (torch.sigmoid(b) - 0.5)

        if self.a_range is not None:
            a = self.a_range * torch.sigmoid(a)
        else:
            a = F.softplus(a)

        if torch.max(theta != theta) or torch.max(a != a) or torch.max(b != b):
            raise ValueError('ValueError:theta,a,b may contains nan!')

        return self.irf(theta, a, b, c, **self.irf_kwargs)

    @classmethod
    def irf(cls, theta, a, b, c, **kwargs):
        return irt3pl(theta, a, b, c, F=torch, **kwargs)

class IRT(CDM):
    def __init__(self, user_num, item_num, value_range=None, a_range=None, use_fairness=True, fairness_lambda=0.5):
        super(IRT, self).__init__()
        self.irt_net = IRTNet(user_num, item_num, value_range, a_range)
        self.use_fairness = use_fairness
        self.fairness_lambda = fairness_lambda
        if use_fairness:
            self.fairness_loss = FairnessLoss()

    def train(self, train_data, test_data=None, *, epoch: int, device="cpu", lr=0.001):
        self.irt_net = self.irt_net.to(device)
        bce_loss = nn.BCELoss()
        trainer = torch.optim.Adam(self.irt_net.parameters(), lr)

        for e in range(epoch):
            losses = []
            bce_losses = []
            fairness_losses = []

            for batch_data in tqdm(train_data, "Epoch %s" % e):
                user_id, item_id, response = batch_data
                predicted_response = self.irt_net(user_id, item_id)
                print("this")

                # 计算BCE损失
                bce_loss_val = bce_loss(predicted_response, response)

                # 如果启用fairness loss
                if self.use_fairness:
                    # 重塑预测和目标以适应fairness loss的计算
                    batch_size = predicted_response.size(0)
                    predictions_reshaped = predicted_response.view(batch_size, -1)
                    targets_reshaped = response.view(batch_size, -1)

                    # 计算fairness loss
                    fairness_loss_val = self.fairness_loss(predictions_reshaped, targets_reshaped)

                    # 组合两种loss
                    loss = (1 - self.fairness_lambda) * bce_loss_val + self.fairness_lambda * fairness_loss_val
                    fairness_losses.append(fairness_loss_val.item())
                else:
                    loss = bce_loss_val

                trainer.zero_grad()
                loss.backward()
                trainer.step()

                losses.append(loss.item())
                bce_losses.append(bce_loss_val.item())

            # 打印训练信息
            log_str = "[Epoch %d] Total Loss: %.6f, BCE Loss: %.6f" % (
                e, float(np.mean(losses)), float(np.mean(bce_losses)))
            if self.use_fairness:
                log_str += ", Fairness Loss: %.6f" % float(np.mean(fairness_losses))
            print(log_str)

            if test_data is not None:
                auc, accuracy = self.eval(test_data, device=device)
                print("[Epoch %d] auc: %.6f, accuracy: %.6f" % (e, auc, accuracy))

    def eval(self, test_data, device="cpu"):
        self.irt_net = self.irt_net.to(device)
        self.irt_net.eval()
        y_pred = []
        y_true = []

        with torch.no_grad():
            for batch_data in tqdm(test_data, "evaluating"):
                user_id, item_id, response = batch_data
                pred = self.irt_net(user_id, item_id)
                y_pred.extend(pred.cpu().numpy())
                y_true.extend(response.cpu().numpy())

        return roc_auc_score(y_true, y_pred), accuracy_score(y_true, np.array(y_pred) >= 0.5)

    def save(self, filepath):
        torch.save(self.irt_net.state_dict(), filepath)
        logging.info("save parameters to %s" % filepath)

    def load(self, filepath):
        self.irt_net.load_state_dict(torch.load(filepath))
        logging.info("load parameters from %s" % filepath)

    def extract_ability_parameters(self, test_data, filepath, device="cpu"):
        self.irt_net = self.irt_net.to(device)
        self.irt_net.eval()

        abilities = []
        processed_user_ids = set()

        with torch.no_grad():
            for batch_data in tqdm(test_data, "Extracting abilities"):
                origin_id, user_id, item_id, response = batch_data
                theta = self.irt_net.theta(user_id).squeeze()

                for i, user in enumerate(user_id.cpu().numpy()):
                    if user not in processed_user_ids:
                        abilities.append([int(origin_id[i]), int(user), float(theta[i].item())])
                        processed_user_ids.add(user)

        df_abilities = pd.DataFrame(abilities, columns=["origin_id", "user_id", "theta"])
        df_abilities.sort_values(by="user_id", inplace=True)
        df_abilities.to_csv(filepath, index=False)

In [2]:
# test_irt.py
# -*- coding: utf-8 -*-

import torch
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
from IRT import IRT
from sklearn.model_selection import train_test_split


train_data = pd.read_csv("./data/a0910/all_virtual_user_data.csv")
valid_data = pd.read_csv("./data/a0910/all_virtual_user_data.csv")
test_data = pd.read_csv("./data/a0910/all_virtual_user_data.csv")

batch_size = 256


def transform(x, y, z, batch_size, **params):
    dataset = TensorDataset(
        torch.tensor(x, dtype=torch.int64),
        torch.tensor(y, dtype=torch.int64),
        torch.tensor(z, dtype=torch.float32)
    )
    return DataLoader(dataset, batch_size=batch_size, **params)


train, valid, test = [
    transform(data["user_id"], data["item_id"], data["score"], batch_size)
    for data in [train_data, valid_data, test_data]
]

# 初始化IRT模型

model = IRT(27424, 17747)

# 训练模型
model.train(train, valid, epoch=10)

# 保存模型
model.save("irt_model.pth")

# 加载模型
model.load("irt_model.pth")

# 评估模型
auc, accuracy = model.eval(test)
print("Test AUC: {}, Test Accuracy: {}".format(auc, accuracy))



#for name, param in model.irt_net.named_parameters():
 #   print(f"Name: {name}, Shape: {param.shape}, Values: {param.data}")


all_virtual_user_data = pd.read_csv('./data/a0910/all_virtual_user_data.csv')

# Transform function to include origin_id
def transform2(x, y, z, origin_ids, batch_size, **params):
    dataset = TensorDataset(
        torch.tensor(origin_ids, dtype=torch.int64),
        torch.tensor(x, dtype=torch.int64),
        torch.tensor(y, dtype=torch.int64),
        torch.tensor(z, dtype=torch.float32)
    )
    return DataLoader(dataset, batch_size=batch_size, **params)

# Prepare test_fairness dataset from all_virtual_user_data
test_fairness = transform2(
    all_virtual_user_data["user_id"],
    all_virtual_user_data["item_id"],
    all_virtual_user_data["score"],
    all_virtual_user_data["origin_id"],  # Include original_id
    batch_size=256
)

# Assuming `model` is your IRT model instance
model.extract_ability_parameters(test_data=test_fairness, filepath="v_ability_parameters.csv")


theta_params = model.irt_net.theta.weight.data.cpu().numpy()
a_params = model.irt_net.a.weight.data.cpu().numpy()
b_params = model.irt_net.b.weight.data.cpu().numpy()
c_params = model.irt_net.c.weight.data.cpu().numpy()

print("Theta parameters (user abilities):", theta_params)
print("a parameters (item discriminations):", a_params)
print("b parameters (item difficulties):", b_params)
print("c parameters (guessing parameters):", c_params)

Epoch 0:   8%|▊         | 634/7995 [00:04<00:57, 127.21it/s]


KeyboardInterrupt: 

In [ ]:

# 加载模型
model.load("irt_model.pth")

# 评估模型
auc, accuracy = model.eval(test)
print("Test AUC: {}, Test Accuracy: {}".format(auc, accuracy))



#for name, param in model.irt_net.named_parameters():
 #   print(f"Name: {name}, Shape: {param.shape}, Values: {param.data}")


all_virtual_user_data = pd.read_csv('/content/Fairness-framework/data/a0910/all_virtual_user_data.csv')

# Transform function to include origin_id
def transform2(x, y, z, origin_ids, batch_size, **params):
    dataset = TensorDataset(
        torch.tensor(origin_ids, dtype=torch.int64),
        torch.tensor(x, dtype=torch.int64),
        torch.tensor(y, dtype=torch.int64),
        torch.tensor(z, dtype=torch.float32)
    )
    return DataLoader(dataset, batch_size=batch_size, **params)

# Prepare test_fairness dataset from all_virtual_user_data
test_fairness = transform2(
    all_virtual_user_data["user_id"],
    all_virtual_user_data["item_id"],
    all_virtual_user_data["score"],
    all_virtual_user_data["origin_id"],  # Include original_id
    batch_size=256
)

# Assuming `model` is your IRT model instance
model.extract_ability_parameters(test_data=test_fairness, filepath="/content/Fairness-framework/IRT/v_ability_parameters.csv")


theta_params = model.irt_net.theta.weight.data.cpu().numpy()
a_params = model.irt_net.a.weight.data.cpu().numpy()
b_params = model.irt_net.b.weight.data.cpu().numpy()
c_params = model.irt_net.c.weight.data.cpu().numpy()

print("Theta parameters (user abilities):", theta_params)
print("a parameters (item discriminations):", a_params)
print("b parameters (item difficulties):", b_params)
print("c parameters (guessing parameters):", c_params)

/content/Fairness-framework/IRT/IRT.py:115: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.irt_net.load_state_dict(torch.load(filepath))
evaluating: 100%|██████████| 799

Test AUC: 0.7405415360219033, Test Accuracy: 0.7081306537525061


Extracting abilities: 100%|██████████| 7995/7995 [00:43<00:00, 183.83it/s]


Ability parameters saved to v_ability_parameters.csv
Theta parameters (user abilities): [[-0.80085695]
 [-0.2261541 ]
 [-0.3283566 ]
 ...
 [-0.11262578]
 [-2.1633432 ]
 [ 0.29993948]]
a parameters (item discriminations): [[ 0.26153198]
 [-0.71217394]
 [-0.89250124]
 ...
 [ 0.6694893 ]
 [-0.2774965 ]
 [-0.61830103]]
b parameters (item difficulties): [[ 1.2625573 ]
 [-0.25795755]
 [ 0.62945867]
 ...
 [-1.1144842 ]
 [-0.34156126]
 [-0.35140184]]
c parameters (guessing parameters): [[-1.7988937e-03]
 [ 1.9305618e-01]
 [ 1.3554970e+00]
 ...
 [ 3.6936715e+00]
 [ 4.7606337e-01]
 [ 2.5666058e+00]]


In [ ]:
# -*- coding: utf-8 -*-
"""公平性度量代码.ipynb

#原来的
"""
import pandas as pd
import numpy as np
from scipy.stats import spearmanr

# 读取数据
file_path = "IRT/v_ability_parameters.csv"
df = pd.read_csv(file_path)

# 定义DCG计算函数
def dcg(ranks, top_k=11):
    gains = np.power(2, ranks)[:top_k] - 1
    discounts = np.log2(np.arange(2, top_k + 2))
    return np.sum(gains / discounts)

# 定义NDCG计算函数
def ndcg(true_ranks, pred_ranks, top_k=11):
    dcg_true = dcg(np.argsort(true_ranks), top_k)
    dcg_pred = dcg(np.argsort(pred_ranks), top_k)
    idcg_true = dcg(np.arange(1, top_k + 1), top_k)
    return dcg_pred / idcg_true if idcg_true > 0 else 0

# 定义Kendall Tau Distance计算函数
def kendall_tau_distance(true_ranks, pred_ranks):
    discordant_pairs = 0
    for i in range(len(true_ranks)):
        for j in range(i + 1, len(true_ranks)):
            if (true_ranks[i] < true_ranks[j]) != (pred_ranks[i] < pred_ranks[j]):
                discordant_pairs += 1
    return discordant_pairs

# 定义Spearman's Rank Correlation Coefficient计算函数
def spearman_rho(true_ranks, pred_ranks):
    return spearmanr(true_ranks, pred_ranks).correlation

# 定义Cosine Similarity计算函数
def cosine_similarity(true_ranks, pred_ranks):
    true_vec = np.array(true_ranks) - 1
    pred_vec = np.array(pred_ranks) - 1
    return np.dot(true_vec, pred_vec) / (np.linalg.norm(true_vec) * np.linalg.norm(pred_vec))

# 定义Mean Reciprocal Rank计算函数
def mean_reciprocal_rank(true_ranks, pred_ranks):
    reciprocal_ranks = []
    for i, true_rank in enumerate(true_ranks):
        rank = np.where(pred_ranks == true_rank)[0]
        if len(rank) > 0:
            reciprocal_ranks.append(1 / (rank[0] + 1))
        else:
            reciprocal_ranks.append(0)
    return np.mean(reciprocal_ranks)

# 中心化user_id和theta
def centerize_ranks(df):
    df['centerized_user_id'] = df.groupby('origin_id')['user_id'].rank(method='min').astype(int)
    df['centerized_theta'] = df.groupby('origin_id')['theta'].rank(method='max', ascending=False).astype(int)
    return df

# 应用中心化
df = centerize_ranks(df)

# 初始化公平性指标的列表
ndcgs = []
kendall_tau_distances = []
spearman_rhos = []
cosine_similarities = []
mean_reciprocal_ranks = []

# 计算每个组的公平性指标
for origin_id in df['origin_id'].unique():
    group_data = df[df['origin_id'] == origin_id]
    true_ranks = group_data['centerized_user_id'].values
    pred_ranks = group_data['centerized_theta'].values
    print(f"Origin ID: {origin_id}")
    print(f"True Ranks: {true_ranks}")
    print(f"Pred Ranks: {pred_ranks}")

    ndcg_score = ndcg(true_ranks, pred_ranks, top_k=11)
    ndcgs.append(ndcg_score)

    ktd = kendall_tau_distance(true_ranks, pred_ranks)
    kendall_tau_distances.append(ktd)

    srho = spearman_rho(true_ranks, pred_ranks)
    spearman_rhos.append(srho)

    cos_sim = cosine_similarity(true_ranks, pred_ranks)
    cosine_similarities.append(cos_sim)

    mrr = mean_reciprocal_rank(true_ranks, pred_ranks)
    mean_reciprocal_ranks.append(mrr)

# 计算平均公平性指标
average_ndcg = np.mean(ndcgs)
average_ktd = np.mean(kendall_tau_distances)
average_srho = np.mean(spearman_rhos)
average_cos_sim = np.mean(cosine_similarities)
average_mrr = np.mean(mean_reciprocal_ranks)

# 打印平均公平性指标
print("Average NDCG:", average_ndcg)
print("Average Kendall Tau Distance:", average_ktd)
print("Average Spearman's Rank Correlation Coefficient:", average_srho)
print("Average Cosine Similarity:", average_cos_sim)
print("Average Mean Reciprocal Rank:", average_mrr)

流式输出内容被截断，只能显示最后 5000 行内容。
Origin ID: 3822
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 9 10  8  5  2  1  6  3 11  7  4]
Origin ID: 856
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 1  3  9 11  2  7  5 10  4  6  8]
Origin ID: 2315
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [10  9  6 11  1  4  2  5  7  8  3]
Origin ID: 334
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 3  5  4  9  8  7  2 11  6 10  1]
Origin ID: 665
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 1  9  2  4  8  5 10  7  3 11  6]
Origin ID: 2723
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 5  6  3  9  2  7  1  4  8 10 11]
Origin ID: 2794
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 3  8  2 11  4  1  7 10  9  6  5]
Origin ID: 643
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 3  9  2  6  1  5 10  8  4 11  7]
Origin ID: 605
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 8 10  2  4  5  7  9  

In [ ]:
# -*- coding: utf-8 -*-
"""公平性度量代码.ipynb
#改进后

"""
import pandas as pd
import numpy as np
from scipy.stats import spearmanr

# 读取数据
file_path = "/content/Fairness-framework/v_ability_parameters.csv"
df = pd.read_csv(file_path)

# 定义DCG计算函数
def dcg(ranks, top_k=11):
    gains = np.power(2, ranks)[:top_k] - 1
    discounts = np.log2(np.arange(2, top_k + 2))
    return np.sum(gains / discounts)

# 定义NDCG计算函数
def ndcg(true_ranks, pred_ranks, top_k=11):
    dcg_true = dcg(np.argsort(true_ranks), top_k)
    dcg_pred = dcg(np.argsort(pred_ranks), top_k)
    idcg_true = dcg(np.arange(1, top_k + 1), top_k)
    return dcg_pred / idcg_true if idcg_true > 0 else 0

# 定义Kendall Tau Distance计算函数
def kendall_tau_distance(true_ranks, pred_ranks):
    discordant_pairs = 0
    for i in range(len(true_ranks)):
        for j in range(i + 1, len(true_ranks)):
            if (true_ranks[i] < true_ranks[j]) != (pred_ranks[i] < pred_ranks[j]):
                discordant_pairs += 1
    return discordant_pairs

# 定义Spearman's Rank Correlation Coefficient计算函数
def spearman_rho(true_ranks, pred_ranks):
    return spearmanr(true_ranks, pred_ranks).correlation

# 定义Cosine Similarity计算函数
def cosine_similarity(true_ranks, pred_ranks):
    true_vec = np.array(true_ranks) - 1
    pred_vec = np.array(pred_ranks) - 1
    return np.dot(true_vec, pred_vec) / (np.linalg.norm(true_vec) * np.linalg.norm(pred_vec))

# 定义Mean Reciprocal Rank计算函数
def mean_reciprocal_rank(true_ranks, pred_ranks):
    reciprocal_ranks = []
    for i, true_rank in enumerate(true_ranks):
        rank = np.where(pred_ranks == true_rank)[0]
        if len(rank) > 0:
            reciprocal_ranks.append(1 / (rank[0] + 1))
        else:
            reciprocal_ranks.append(0)
    return np.mean(reciprocal_ranks)

# 中心化user_id和theta
def centerize_ranks(df):
    df['centerized_user_id'] = df.groupby('origin_id')['user_id'].rank(method='min').astype(int)
    df['centerized_theta'] = df.groupby('origin_id')['theta'].rank(method='max', ascending=False).astype(int)
    return df

# 应用中心化
df = centerize_ranks(df)

# 初始化公平性指标的列表
ndcgs = []
kendall_tau_distances = []
spearman_rhos = []
cosine_similarities = []
mean_reciprocal_ranks = []

# 计算每个组的公平性指标
for origin_id in df['origin_id'].unique():
    group_data = df[df['origin_id'] == origin_id]
    true_ranks = group_data['centerized_user_id'].values
    pred_ranks = group_data['centerized_theta'].values
    print(f"Origin ID: {origin_id}")
    print(f"True Ranks: {true_ranks}")
    print(f"Pred Ranks: {pred_ranks}")

    ndcg_score = ndcg(true_ranks, pred_ranks, top_k=11)
    ndcgs.append(ndcg_score)

    ktd = kendall_tau_distance(true_ranks, pred_ranks)
    kendall_tau_distances.append(ktd)

    srho = spearman_rho(true_ranks, pred_ranks)
    spearman_rhos.append(srho)

    cos_sim = cosine_similarity(true_ranks, pred_ranks)
    cosine_similarities.append(cos_sim)

    mrr = mean_reciprocal_rank(true_ranks, pred_ranks)
    mean_reciprocal_ranks.append(mrr)

# 计算平均公平性指标
average_ndcg = np.mean(ndcgs)
average_ktd = np.mean(kendall_tau_distances)
average_srho = np.mean(spearman_rhos)
average_cos_sim = np.mean(cosine_similarities)
average_mrr = np.mean(mean_reciprocal_ranks)

# 打印平均公平性指标
print("Average NDCG:", average_ndcg)
print("Average Kendall Tau Distance:", average_ktd)
print("Average Spearman's Rank Correlation Coefficient:", average_srho)
print("Average Cosine Similarity:", average_cos_sim)
print("Average Mean Reciprocal Rank:", average_mrr)

流式输出内容被截断，只能显示最后 5000 行内容。
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 5  7 10  1  3 11  6  4  9  2  8]
Origin ID: 856
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 2  4 11  9  5 10  8  6  3  1  7]
Origin ID: 2315
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 8 11  3  2  7 10  1  9  5  6  4]
Origin ID: 334
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 4  5  7  9  2  3  1  8 11  6 10]
Origin ID: 665
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 1 11  3  6 10  5  4  8  9  7  2]
Origin ID: 2723
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 4  1  5  2  3 11  6  8  9 10  7]
Origin ID: 2794
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 2  1  9  6 11  3  4  5 10  7  8]
Origin ID: 643
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [10  3  1  9  6  4  7  8  5  2 11]
Origin ID: 605
True Ranks: [ 1  2  3  4  5  6  7  8  9 10 11]
Pred Ranks: [ 1  3  4  7  5 10  9  2 11  6  8]
Orig